<a href="https://colab.research.google.com/github/HackElite-FYP/Legal-Research-Platform/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
from google.colab import userdata

GH_UNAME = userdata.get('GH_UNAME')
GH_APIKEY = userdata.get('GH_APIKEY')
PRIMARY_REPO_NAME = 'Legal-Research-Platform'
LOCAL_REPO_DIR = '/content/drive/MyDrive/FYP/GitHub/Legal-Research-Platform'

%cd {LOCAL_REPO_DIR}

/content/drive/MyDrive/FYP/GitHub/Legal-Research-Platform


In [ ]:
%ls -a

.git/  .gitignore  README.md
